In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics, preprocessing, model_selection
import xgboost as xgb
import logging
import sys

In [2]:
# logging.basicConfig(filename='./log.log', level=logging.INFO,
#                             format='%(asctime)s %(levelname)s %(name)s %(message)s')
# logger = logging.getLogger(__name__)
# sys.stdout=open("./log.log","w")

In [3]:
def make_submission_prediction(model, X_train, y_train, scaler, csv=False):
    data = pd.read_csv('./test.csv')
    id_code = np.array(data.ID_code)
    X_test = np.array(scaler.transform(data.drop(['ID_code'], 1)))
    fpr, tpr, thresholds = metrics.roc_curve(y_train, model.predict(X_train))
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]
    pred = model.predict(X_test)
    pred = (pred > optimal_threshold).astype(int)
    pred = pd.DataFrame({'ID_code': id_code, 'target': pred})
    if csv:
        pred.to_csv('prediction.csv', index=False)
    return pred

In [4]:
def validate_prediction(model, X_train, y_train, X_val, y_val, scaler):
    X_test = np.array(scaler.transform(X_val))
    fpr, tpr, thresholds = metrics.roc_curve(y_train, model.predict(X_train))
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]
    pred = model.predict(X_test)
    pred = (pred > optimal_threshold).astype(int)
    print(metrics.roc_auc_score(y_val, pred))
    return pred

In [5]:
data = './train.csv'
data = pd.read_csv(data)
data.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [6]:
X = data.drop(['target', 'ID_code'], 1)
y = data.target
X_train, X_val, y_train, y_val = model_selection.train_test_split(X, y, test_size=0.1)

In [7]:
prop = (y == 0).sum().astype(float)/(y == 1).sum()
prop

8.951238929246692

In [8]:
QuantileScaler = preprocessing.QuantileTransformer(n_quantiles=10000)
QuantileScaler.fit(X_train)
X_train = QuantileScaler.transform(X_train)

## Based data prepartion finished

In [9]:
# params = {'objective':'binary:logistic', 'eval_metric': 'auc', 'n_jobs': 12, 'tree_method': 'gpu_hist', 'verbosity':1, 
#           'max_depth': 16, 'eta': 0.01, 'subsample': 0.5, 'min_obs_node': 1, 
#           'booster': 'gbtree', 'scale_pos_weight' : 1/prop,
#          }

In [10]:
# booster_gpu = xgb.XGBRegressor(**params)

In [11]:
# booster_gpu.fit(X_train, np.array(y_train))

In [12]:
# validate_prediction(booster_gpu, X_train, y_train, X_val, y_val, QuantileScaler)

In [13]:
# make_submission_prediction(booster_gpu, X_train, y_train, QuantileScaler, True)

### Model above without any fine-tunning reached 0.63943 in submission validation. Next we will do some EDA and model fine-tunning

In [14]:
# import seaborn as sns
# corr = X.corr()
# ax = sns.heatmap(
#     corr, 
#     vmin=-1, vmax=1, center=0,
#     cmap=sns.diverging_palette(20, 220, n=200),
#     square=True
# )
# ax.set_xticklabels(
#     ax.get_xticklabels(),
#     rotation=45,
#     horizontalalignment='right'
# )

In [15]:
# standardize the validation data before calculating
def auc(estimator, X, y):
    prediction = estimator.predict(X)
    fpr, tpr, thresholds = metrics.roc_curve(y, prediction)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]
    pred = (prediction > optimal_threshold).astype(int)
    return metrics.roc_auc_score(y, pred)

In [16]:
params = {'objective':'binary:logistic', 'eval_metric': 'auc', 'n_jobs': 12, 'tree_method': 'gpu_hist', 
          'verbosity':1, 'booster': 'gbtree', 'scale_pos_weight' : 1/prop,
          'eta': 0.01, 'subsample': 0.5, 'min_obs_node': 1, 
         }
booster = xgb.XGBRegressor(**params)

In [17]:
grid = {'max_bin': [100, 250, 1000],
        'grow_policy': ['lossguide', 'depthwise'],
        'max_leaves': [0, 10, 100],
        'alpha': np.exp(np.linspace(-10, 10, 10)),
        'lambda': np.exp(np.linspace(-10, 10, 10)),
        'colsample_bytree': [0.2, 0.5, 0.8, 1],
        'colsample_bylevel': [0.2, 0.5, 0.8, 1],
        'colsample_bynode': [0.2, 0.5, 0.8, 1],
        'subsample': [0.2, 0.5, 0.8, 1],
        'max_delta_step': [0, 1, 10],
        'min_child_weight': [1, 5, 10, 100],
        'max_depth': [3, 5, 16, 50, 100], 
        'gamma': np.exp(np.linspace(-10, 10, 10)),
        'eta': np.exp(np.linspace(-5, 0, 10)),
        'n_estimators': [100, 500, 1000, 5000],
        'learning_rate': [0.01, 0.1]}

In [18]:
rs = model_selection.RandomizedSearchCV(cv=3, n_jobs=1, verbose=100, scoring=auc,
                                       estimator=booster, param_distributions=grid, n_iter=5)

In [19]:
%time rs.fit(X_train, np.array(y_train))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] subsample=1, n_estimators=1000, min_child_weight=100, max_leaves=10, max_depth=100, max_delta_step=1, max_bin=250, learning_rate=0.01, lambda=2386.9645567479115, grow_policy=lossguide, gamma=258.67063051550025, eta=1.0, colsample_bytree=0.2, colsample_bynode=1, colsample_bylevel=1, alpha=0.0004189421234483841 
[CV]  subsample=1, n_estimators=1000, min_child_weight=100, max_leaves=10, max_depth=100, max_delta_step=1, max_bin=250, learning_rate=0.01, lambda=2386.9645567479115, grow_policy=lossguide, gamma=258.67063051550025, eta=1.0, colsample_bytree=0.2, colsample_bynode=1, colsample_bylevel=1, alpha=0.0004189421234483841, score=0.750, total=  10.2s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.2s remaining:    0.0s
[CV] subsample=1, n_estimators=1000, min_child_weight=100, max_leaves=10, max_depth=100, max_delta_step=1, max_bin=25

[CV]  subsample=1, n_estimators=5000, min_child_weight=1, max_leaves=100, max_depth=5, max_delta_step=10, max_bin=250, learning_rate=0.01, lambda=3.0377317775174815, grow_policy=depthwise, gamma=258.67063051550025, eta=0.06217652402211632, colsample_bytree=0.5, colsample_bynode=0.2, colsample_bylevel=0.8, alpha=0.0004189421234483841, score=0.813, total=  54.7s
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:  3.9min remaining:    0.0s
[CV] subsample=0.2, n_estimators=1000, min_child_weight=5, max_leaves=0, max_depth=50, max_delta_step=0, max_bin=250, learning_rate=0.01, lambda=3.0377317775174815, grow_policy=lossguide, gamma=28.03162489452615, eta=0.32919298780790573, colsample_bytree=1, colsample_bynode=0.5, colsample_bylevel=1, alpha=0.32919298780790573 
[CV]  subsample=0.2, n_estimators=1000, min_child_weight=5, max_leaves=0, max_depth=50, max_delta_step=0, max_bin=250, learning_rate=0.01, lambda=3.0377317775174815, grow_policy=lossguide, gamma=28.03162489452615, eta=0.329192987

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1, eta=0.01,
                                          eval_metric='auc', gamma=0,
                                          importance_type='gain',
                                          learning_rate=0.1, max_delta_step=0,
                                          max_depth=3, min_child_weight=1,
                                          min_obs_node=1, missing=None,
                                          n_estimators=100, n_jobs=12,
                                          nt...
                                        'learning_rate': [0.01, 0.1],
                                        'max_bin': [100, 250, 1000],
                                        'max_delt

In [20]:
# sys.stdout.close()

In [21]:
rs.best_params_

{'subsample': 1,
 'n_estimators': 5000,
 'min_child_weight': 1,
 'max_leaves': 100,
 'max_depth': 5,
 'max_delta_step': 10,
 'max_bin': 250,
 'learning_rate': 0.01,
 'lambda': 3.0377317775174815,
 'grow_policy': 'depthwise',
 'gamma': 258.67063051550025,
 'eta': 0.06217652402211632,
 'colsample_bytree': 0.5,
 'colsample_bynode': 0.2,
 'colsample_bylevel': 0.8,
 'alpha': 0.0004189421234483841}

In [23]:
validate_prediction(rs.best_estimator_, X_train, y_train, X_val, y_val, QuantileScaler)

0.8060829090420257


array([1, 0, 0, ..., 1, 0, 0])

In [25]:
make_submission_prediction(rs.best_estimator_, X_train, y_train, QuantileScaler, csv=True)

,ID_code,target
0,test_0,0
1,test_1,1
2,test_2,1
3,test_3,0
4,test_4,0
...,...,...
199995,test_199995,0
199996,test_199996,0
199997,test_199997,0
199998,test_199998,0


In [39]:
pd.DataFrame([rs.best_params_]).transpose()

,0
subsample,1
n_estimators,5000
min_child_weight,1
max_leaves,100
max_depth,5
max_delta_step,10
max_bin,250
learning_rate,0.01
lambda,3.03773
grow_policy,depthwise
